# Final Data File Creator

This notebook creates a complete data file that we intend to use for all our future location and mode choice models.

In [216]:
import pandas as pd
import math
import csv

hh_df = pd.read_csv('SMTO_2015/SMTO_2015_Households.csv')
ps_df = pd.read_csv('SMTO_2015/SMTO_2015_Respondents.csv')
campus_info = pd.read_csv('SMTO_2015/Campus_Info.csv')

# Load relevant columns
df = ps_df[['pscampusattend', 'personstatusgrad', 'personstatustime','psmainmodefalltypical', 'pscmpgender','psdrivinglicenseownerflag','psworknumhoursperweek','psage','psexpfactor']]
df = df.join(hh_df[['HmTTS2006', 'hhlivingsituation', 'hhcarnumber','hhnumyoungerthan18','hhnumolderorequalto18','hhincomelevel']])

df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'hhlivingsituation': 'Family', 'psmainmodefalltypical': 'Mode', 
                       'personstatusgrad': 'Level', 'personstatustime': 'Status', 'psuniversityinvolvednumyears': 'Years', 'hhcarnumber': 'Cars',
                       'hhincomelevel': 'Income', 'pscmpgender': 'Gender', 'psdrivinglicenseownerflag': 'Licence', 'psworknumhoursperweek': 'Work',
                       'hhnumyoungerthan18': 'Children', 'hhnumolderorequalto18':'Adults', 'psage': 'Age','psexpfactor': 'Exp_SMTO'})


mode_name_to_num = {"Car - Driver alone": "Auto", "Car - Driver with passenger(s)": "Auto", "Car - Passenger": "Auto", "Taxi": "Auto", 
                    "Transit Bus": "Transit", "Streetcar": "Transit", "Subway/RT": "Transit", "GO Bus": "Transit", "GO Train": "Transit", 
                    "Walk": "Active", "Bicycle": "Active"}
print(df.shape)

df.replace({'Mode': mode_name_to_num}, inplace=True)
df = df[(df['Mode'] == "Auto") | (df['Mode'] == "Transit") | (df['Mode'] == "Active")]
print(df.shape)

df['Work'].fillna('Unknown', inplace=True) # Fill 11,000 Work NaNs with "Unknowns"
df = df.dropna() # Remove rows with missing data
print(df.shape)

# Convert Campus column to numerical column
df['CampusZone'] = df["Campus"].apply(lambda x: campus_info.Zone[campus_info['School'] == x].tolist()[0])
print(df.shape)

# Convert columns to numerical
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')

print(df.shape)
df.head()

(15226, 15)
(15066, 15)
(14839, 15)
(14839, 16)
(14839, 16)


,Campus,Level,Status,Mode,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,CampusZone
0,Scarborough (UTSC),UG,FT,Transit,Female,0,Unknown,20,9.70,261,Live with family/parents,1,3,4,Unknown,566
1,Downtown Toronto (St. George),Grad,FT,Active,Female,1,Unknown,25,5.79,71,Live with partner,0,0,2,"$ 90,000 - 119,999",69
2,Downtown Toronto (St. George),UG,FT,Transit,Female,1,Unknown,23,9.06,3714,Live with family/parents,1,0,4,Unknown,69
3,Downtown Toronto (St. George),UG,FT,Active,Male,1,Unknown,20,14.67,74,Live with roommates,0,0,4,Unknown,69
4,Downtown Toronto (St. George),Grad,FT,Active,Male,1,Unknown,27,9.11,71,Live with partner,0,0,2,"$ 30,000 - 59,999",69


In [217]:
df['Campus'].value_counts()

Downtown Toronto (St. George)    6004
Keele                            3173
RyersonU                         2882
Scarborough (UTSC)               1082
Mississauga (UTM)                 904
OCADu                             465
Glendon                           329
Name: Campus, dtype: int64

In [218]:
# Column Transformation functions

def inc_transform(x):
    if x == "Less than $ 30,000":
        return "Low"
    elif x == "$ 30,000 - 59,999":
        return "Low"
    elif x == "$ 60,000 - 89,999":
        return "Low"
    elif x == "$ 90,000 - 119,999":
        return "High"
    elif x == "$ 120,000 - 149,999":
        return "High"
    elif x == "$ 150,000 - 179,999":
        return "High"
    elif x == "$ 180,000 - 209,999":
        return "High"
    elif x == "$ 210,000 - 239,999":
        return "High"
    elif x == "$ 240,000 +":
        return "High"
    else:
        return "Unknown"
   
    
def work_transform(x):
    if x == "Yes, I work > 40 hours per week":
        return "FT"
    elif x == "Yes, I work 31-40 hours per week":
        return "FT"
    elif x == "Yes, I work part time (21-30 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (11-20 hours per week)":
        return "PT"
    elif x == "Yes, I work part time (<10 hours per week)":
        return "PT"
    else:
        return "Other"
    
def fam_transform(x):
    if x == "Live with family/parents":
        return "Family"
    elif x == "Live with roommates":
        return "Roommates"
    else:
        return "Other"
'''
def car_transform(x):
    if x == 0:
        return 0
    elif x == 1: 
        return 1
    else:
        return 2
'''

'\ndef car_transform(x):\n    if x == 0:\n        return 0\n    elif x == 1: \n        return 1\n    else:\n        return 2\n'

In [219]:
df['Work'] = df["Work"].apply(lambda x: work_transform(x))
df['Family'] = df["Family"].apply(lambda x: fam_transform(x))
#df['Cars'] = df["Cars"].apply(lambda x: car_transform(x))
df['Income'] = df["Income"].apply(lambda x: inc_transform(x))
df.head()

,Campus,Level,Status,Mode,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,CampusZone
0,Scarborough (UTSC),UG,FT,Transit,Female,0,Other,20,9.70,261,Family,1,3,4,Unknown,566
1,Downtown Toronto (St. George),Grad,FT,Active,Female,1,Other,25,5.79,71,Other,0,0,2,High,69
2,Downtown Toronto (St. George),UG,FT,Transit,Female,1,Other,23,9.06,3714,Family,1,0,4,Unknown,69
3,Downtown Toronto (St. George),UG,FT,Active,Male,1,Other,20,14.67,74,Roommates,0,0,4,Unknown,69
4,Downtown Toronto (St. George),Grad,FT,Active,Male,1,Other,27,9.11,71,Other,0,0,2,Low,69


In [220]:
def row_to_segment(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return 3 if x.Status == 'PT' else (not x.Family) + 1
    else:
        return 6 if x.Status == 'PT' else (4 + (not x.Family))

df['Segment'] = df.apply(row_to_segment, axis=1)

df.head()

,Campus,Level,Status,Mode,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,Family,Cars,Children,Adults,Income,CampusZone,Segment
0,Scarborough (UTSC),UG,FT,Transit,Female,0,Other,20,9.70,261,Family,1,3,4,Unknown,566,1
1,Downtown Toronto (St. George),Grad,FT,Active,Female,1,Other,25,5.79,71,Other,0,0,2,High,69,4
2,Downtown Toronto (St. George),UG,FT,Transit,Female,1,Other,23,9.06,3714,Family,1,0,4,Unknown,69,1
3,Downtown Toronto (St. George),UG,FT,Active,Male,1,Other,20,14.67,74,Roommates,0,0,4,Unknown,69,1
4,Downtown Toronto (St. George),Grad,FT,Active,Male,1,Other,27,9.11,71,Other,0,0,2,Low,69,4


In [221]:
# Dataframe with walk distances
df_path = pd.read_csv('../../LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

# Dataframe with AutoTravelTimes
df_att = pd.read_csv('../../LoS/Auto_Travel_Times.csv')
AutoTravelTimes = list(df_att['Data'])

# Dataframe with TransitTravelTimes
df_ttt = pd.read_csv('../../LoS/Transit_Travel_Times.csv')
TransitTravelTimes = list(df_ttt['Data'])

# Function for distance lookup
not_found = set()
def find_distance(origin, destination, info_num):
    # mode_num: 0 - to find Walking Distance
    #           1 - to find Walking Time
    #           2 - to find Auto Time  
    #           3 - to find Transit Time
    
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return -1
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return -1
    
    if info_num == 0:
        return dists[i*2392 + j] / 1000
    elif info_num == 1:
        return dists[i*2392 + j] * 15 / 1000 
    elif info_num == 2:
        return AutoTravelTimes[i*2392 + j]
    elif info_num == 3:
        return TransitTravelTimes[i*2392 + j]
    else:
        print("ERROR: Enter correct info_num!")
        return False


# List of campus' TTS zones from Joven's MOE data
campus_zones = list(campus_info['Zone'])
school_codes = list(campus_info['Code'])

# Load distances into dataframe
for i in range(len(campus_zones)):
    df["Dist." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],0))
for i in range(len(campus_zones)):
    df["Time.Active." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],1))
for i in range(len(campus_zones)):
    df["Time.Auto." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],2))
for i in range(len(campus_zones)):
    df["Time.Transit." + school_codes[i]] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i],3))
    
print("# of zones not found:", len(not_found))

df.head()

# of zones not found: 123


,Campus,Level,Status,Mode,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,...,Time.Auto.YG,Time.Auto.RY,Time.Auto.OC,Time.Transit.SG,Time.Transit.SC,Time.Transit.MI,Time.Transit.YK,Time.Transit.YG,Time.Transit.RY,Time.Transit.OC
0,Scarborough (UTSC),UG,FT,Transit,Female,0,Other,20,9.70,261,...,15.30110,24.897050,27.112270,73.276483,75.468478,129.369386,197.858689,64.076936,75.379996,81.104413
1,Downtown Toronto (St. George),Grad,FT,Active,Female,1,Other,25,5.79,71,...,19.49373,5.748302,5.553657,24.128386,78.020223,77.135050,144.674281,66.918003,31.749210,27.804764
2,Downtown Toronto (St. George),UG,FT,Transit,Female,1,Other,23,9.06,3714,...,54.99243,51.372310,48.720940,155.551170,124.359872,42.439563,85.084262,186.874439,147.452825,146.732427
3,Downtown Toronto (St. George),UG,FT,Active,Male,1,Other,20,14.67,74,...,22.30349,4.768110,3.258073,16.675728,88.246135,83.728866,152.561525,77.003957,21.273948,19.897948
4,Downtown Toronto (St. George),Grad,FT,Active,Male,1,Other,27,9.11,71,...,19.49373,5.748302,5.553657,24.128386,78.020223,77.135050,144.674281,66.918003,31.749210,27.804764


In [222]:
for col in df.columns:
    print(col)

Campus
Level
Status
Mode
Gender
Licence
Work
Age
Exp_SMTO
HomeZone
Family
Cars
Children
Adults
Income
CampusZone
Segment
Dist.SG
Dist.SC
Dist.MI
Dist.YK
Dist.YG
Dist.RY
Dist.OC
Time.Active.SG
Time.Active.SC
Time.Active.MI
Time.Active.YK
Time.Active.YG
Time.Active.RY
Time.Active.OC
Time.Auto.SG
Time.Auto.SC
Time.Auto.MI
Time.Auto.YK
Time.Auto.YG
Time.Auto.RY
Time.Auto.OC
Time.Transit.SG
Time.Transit.SC
Time.Transit.MI
Time.Transit.YK
Time.Transit.YG
Time.Transit.RY
Time.Transit.OC


In [223]:
# Load enrollment data
enrollment_df = pd.read_csv('Enrolment/Joven_Enrollment.csv').set_index('School')
def get_log_enrollment(level, school):
    return math.log1p(enrollment_df.loc[school][level]) # Level: UG, Grad, Total
    
for level in ('Total', 'UG', 'Grad'):
    for i in range(len(campus_zones)):
        df[level + "." + school_codes[i]] = get_log_enrollment(level, school_codes[i])

In [224]:
df.head()

,Campus,Level,Status,Mode,Gender,Licence,Work,Age,Exp_SMTO,HomeZone,...,UG.YG,UG.RY,UG.OC,Grad.SG,Grad.SC,Grad.MI,Grad.YK,Grad.YG,Grad.RY,Grad.OC
0,Scarborough (UTSC),UG,FT,Transit,Female,0,Other,20,9.70,261,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
1,Downtown Toronto (St. George),Grad,FT,Active,Female,1,Other,25,5.79,71,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
2,Downtown Toronto (St. George),UG,FT,Transit,Female,1,Other,23,9.06,3714,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
3,Downtown Toronto (St. George),UG,FT,Active,Male,1,Other,20,14.67,74,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055
4,Downtown Toronto (St. George),Grad,FT,Active,Male,1,Other,27,9.11,71,...,7.758761,10.166928,8.107419,9.591376,5.624018,6.385194,8.26359,4.762174,7.665285,5.159055


In [225]:
uni_info = pd.read_csv('Uni_Info.csv')
uni_info = uni_info.set_index('University')
uni_info.head()

,Code,Tuition,Domestic%_UG,Domestic%_Grad,Admission
University,,,,,
Downtown Toronto (St. George),SG,7519,0.8080,0.8369,0.893
Scarborough (UTSC),SC,7813,0.8378,0.8406,0.841
Mississauga (UTM),MI,7670,0.8282,0.6981,0.830
Keele,YK,7339,0.8921,0.8077,0.817
Glendon,YG,7339,0.8921,0.8077,0.817


In [226]:
# One column per campus tuition
for i in range(len(campus_zones)):
    df["Tuition." + school_codes[i]] = uni_info['Tuition'][i]
    
# One single tuition col
#df['Tuition'] = df['Campus'].apply(lambda x: uni_info.loc[x]['Tuition'])

In [227]:
# Domestic Percentage Column
for i in range(len(campus_zones)):
    df["Domestic%." + school_codes[i]] = df['Level'].apply(lambda x: uni_info['Domestic%_' + ('Grad' if (x == 'Grad') else 'UG')][i])

In [228]:
for i in range(len(campus_zones)):
    df["Admission_Avg." + school_codes[i]] = uni_info['Admission'][i]

In [229]:
list(uni_info.index)

['Downtown Toronto (St. George)',
 'Scarborough (UTSC)',
 'Mississauga (UTM)',
 'Keele',
 'Glendon',
 'RyersonU',
 'OCADu']

In [230]:
exp_factors = {}
schoolnames = list(uni_info.index)
for i in range(len(campus_zones)):
    exp_factors[schoolnames[i]] = [enrollment_df.loc[school_codes[i]]['UG'] / enrollment_df['UG'].sum() / df[df['Level'] == 'UG']['Campus'].value_counts(normalize=True)[i]]
    exp_factors[schoolnames[i]].append(enrollment_df.loc[school_codes[i]]['Grad'] / enrollment_df['Grad'].sum() / df[df['Level'] == 'Grad']['Campus'].value_counts(normalize=True)[i]) 
    
print(exp_factors)
    
def load_exp_factors(x):
    if x.Level == 'Other':
        return x.Exp_SMTO
    return exp_factors[x.Campus][0 if x.Level == 'UG' else 1]


df['Exp'] = df.apply(load_exp_factors, axis = 1)


{'Downtown Toronto (St. George)': [0.9192704341030282, 0.9860853627103758], 'Scarborough (UTSC)': [0.38370546842060904, 0.06958687513875882], 'Mississauga (UTM)': [0.4370827811077548, 0.32598537542357764], 'Keele': [3.062795384490451, 10.60622128743012], 'Glendon': [0.23379599973493054, 0.31717495987158906], 'RyersonU': [5.332428849357958, 6.261268652279888], 'OCADu': [0.918397217184961, 1.613860825228968]}


In [231]:
# Remove columns we don't need anymore:
del df['CampusZone']
del df['HomeZone']
del df['Exp_SMTO']

In [232]:
for i in school_codes:
    df = df[df['Time.Active.' + i] != -1]

In [233]:
df.columns

Index(['Campus', 'Level', 'Status', 'Mode', 'Gender', 'Licence', 'Work', 'Age',
       'Family', 'Cars', 'Children', 'Adults', 'Income', 'Segment', 'Dist.SG',
       'Dist.SC', 'Dist.MI', 'Dist.YK', 'Dist.YG', 'Dist.RY', 'Dist.OC',
       'Time.Active.SG', 'Time.Active.SC', 'Time.Active.MI', 'Time.Active.YK',
       'Time.Active.YG', 'Time.Active.RY', 'Time.Active.OC', 'Time.Auto.SG',
       'Time.Auto.SC', 'Time.Auto.MI', 'Time.Auto.YK', 'Time.Auto.YG',
       'Time.Auto.RY', 'Time.Auto.OC', 'Time.Transit.SG', 'Time.Transit.SC',
       'Time.Transit.MI', 'Time.Transit.YK', 'Time.Transit.YG',
       'Time.Transit.RY', 'Time.Transit.OC', 'Total.SG', 'Total.SC',
       'Total.MI', 'Total.YK', 'Total.YG', 'Total.RY', 'Total.OC', 'UG.SG',
       'UG.SC', 'UG.MI', 'UG.YK', 'UG.YG', 'UG.RY', 'UG.OC', 'Grad.SG',
       'Grad.SC', 'Grad.MI', 'Grad.YK', 'Grad.YG', 'Grad.RY', 'Grad.OC',
       'Tuition.SG', 'Tuition.SC', 'Tuition.MI', 'Tuition.YK', 'Tuition.YG',
       'Tuition.RY', 'Tuition.OC'

In [234]:
df.shape

(14661, 85)

In [235]:
df.to_csv('SMTO_2015/SMTO_2015_Complete_Input.csv', index = False)